In [1]:
from tqdm import tqdm
from collections import Counter
from itertools import combinations
from nltk.stem import PorterStemmer
import joblib
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

import imp
import copy
import pickle
import multiprocessing

import numpy as np
import pandas as pd
import utils as my_utils
import ELJST_script_BTM as lda
import matplotlib.pyplot as plt

In [2]:
min_df = 5
max_df = .5
maxIters = 5

beta = .01
gamma = 10
n_topics = 5
lambda_param = 1.0

In [3]:
dataset_name = "imdb_reviews_47405"

In [4]:
dataset_ = pd.read_pickle("resources/"+ dataset_name + "_" + "dataset")

In [5]:
n_sentiment = dataset_.sentiment.unique().shape[0]

In [6]:
embedding_name = "fasttext_0.6"

In [9]:
similar_words_ = pickle.load(open("resources/"+ dataset_name + "_" +embedding_name + ".pickle","rb"))

In [10]:
%%time
for s in similar_words_:
    for i in s.keys():
        k = list(set(s[i]))
        if i in k:
            k.remove(i)
        s[i] = k

CPU times: user 3.33 s, sys: 32.5 ms, total: 3.36 s
Wall time: 3.36 s


In [27]:
_, dataset, _, similar_words = train_test_split(dataset_, similar_words_, test_size=0.1,
                                                random_state=42, stratify=dataset_.sentiment)

In [28]:
dataset.shape

(4741, 2)

In [32]:
dataset.head()

,text,sentiment
22073,campfire tale excellent peace work everything ...,1
421,interest hour scenic mountain footage feature ...,0
44769,light head comedy wonderful family call pecker...,1
10176,movie worst disaster sick movie waste time wat...,0
40830,excite movie poster funniest movie dorm septem...,1


In [29]:
alpha = 0.1/n_topics * np.ones(n_topics)
gamma = [gamma/(n_topics*n_sentiment)]*n_sentiment

In [34]:
imp.reload(lda)

<module 'ELJST_script_BTM' from '/home/william18026/Embedding-LJST/ELJST_script_BTM.py'>

In [35]:
sampler = lda.SentimentLDAGibbsSampler(n_topics, alpha, beta, gamma, numSentiments=n_sentiment, 
                                       SentimentRange = n_sentiment, max_df = max_df, min_df = min_df, 
                                       lambda_param = lambda_param)

In [36]:
%%time
sampler._initialize_(reviews = dataset.text.tolist(), labels = dataset.sentiment.tolist(), skipgramwindow=5)

CPU times: user 2min 31s, sys: 1.11 s, total: 2min 32s
Wall time: 2min 32s


In [ ]:
sampler.run(name=embedding_name, reviews=dataset.text.tolist(), labels=dataset.sentiment.tolist(), 
            similar_words=similar_words, mrf=True, maxIters=maxIters, debug=True)

  0%|          | 0/4741 [00:00<?, ?it/s]

** fasttext_0.6 0


 52%|█████▏    | 2453/4741 [09:02<09:08,  4.17it/s]

In [ ]:
joblib.dump(sampler, "dumps/BTM_tester_" + embedding_name + "_" + str(lambda_param))

In [ ]:
s = joblib.load("dumps/BTM_tester_" + embedding_name + "_" + str(lambda_param))

# Evaluations

In [ ]:
plt.plot(sampler.loglikelihood_history)

In [ ]:
silhouette_score(euclidean_distances(sampler.wordOccuranceMatrix),
                 sampler.dt_distribution.argmax(axis=1), metric='precomputed')

In [ ]:
davies_bouldin_score(sampler.wordOccuranceMatrix, sampler.dt_distribution.argmax(axis=1))

In [ ]:
my_utils.coherence_score(sampler.wordOccuranceMatrix, list(sampler.getTopKWords(5).values()), sampler.vocabulary)

In [ ]:
%%time
my_utils.get_hscore_multi(sampler.dt_distribution, sampler.wordOccuranceMatrix, n_topics, 500)

In [ ]:
sampler.loglikelihood()

In [ ]:
sampler.perplexity()

## Appendix

In [ ]:
# joblib.dump(sampler, "resources/BTM_sampler_0.4_1")